In [8]:
!pip install git+git://github.com/huggingface/transformers/
!pip install --upgrade tensorflow
!python -m pip install --upgrade pip
!pip install -U sentence-transformers
!pip install pyngrok
!pip install flask_ngrok
!pip install transformers torch
!pip install cleantext

  Cloning git://github.com/huggingface/transformers/ to /tmp/pip-req-build-mns5difl
  Running command git clone -q git://github.com/huggingface/transformers/ /tmp/pip-req-build-mns5difl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [128]:
import pandas as pd
import io
import re
import random
import numpy as np
import logging
import math
import os
from dataclasses import dataclass, field
from typing import Optional
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import (
CONFIG_MAPPING,
MODEL_WITH_LM_HEAD_MAPPING,
AutoConfig,
AutoModelWithLMHead,
GPT2LMHeadModel,
AutoTokenizer,
DataCollatorForLanguageModeling,
DataCollatorForPermutationLanguageModeling,
HfArgumentParser,
LineByLineTextDataset,
PreTrainedTokenizer,
TextDataset,
Trainer,
TrainingArguments,
set_seed,
)
model_name='bert-base-nli-mean-tokens'
modelBert=SentenceTransformer(model_name)
import cleantext
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyngrok import ngrok
vectorizer = TfidfVectorizer()
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
cd /content/drive/MyDrive/Colab Notebooks/webpage

/content/drive/MyDrive/Colab Notebooks/webpage


## Pre-Processing  dataset and Training Model

In [ ]:
!mkdir wikiPlots
!unzip '/content/gdrive/MyDrive/WikiMoviePlots.zip' -d 'wikiPlots'
import pandas as pd
import io
import numpy as np

In [ ]:
df = pd.read_csv("/content/wikiPlots/wiki_movie_plots_deduped.csv", na_values = ['no info', '.'])
df.reset_index(drop=True, inplace=True)
genre_selection=df.loc[df['Genre'] == "comedy"]
tmp=genre_selection['Plot'].tolist()
tmp= np.array([('<BOS> <comedy> '+xi +' <EOS>') for xi in tmp])
np.savetxt(r'/content/wikiPlots/comedyText.txt',tmp, fmt='%s')

In [ ]:
!split -b $(expr $(cat '/content/wikiPlots/comedyText.txt' | wc -m) \* 80 / 100) '/content/wikiPlots/comedyText.txt' 'comedy_train.txt'

## GPT2 Story Generator

In [ ]:
# import torch
# import gc
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.memory_summary(device=None, abbreviated=False)


In [ ]:
def genGpt2(f1,f2,f3,genreList,s1,s2,s3):
  # Setup logging
  logger = logging.getLogger(__name__)

  # Get access to model types and model configs to select GPT2 model and config
  MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
  MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


  @dataclass
  class ModelArguments:
      """
      Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
      """

      model_name_or_path: Optional[str] = field(
          default=None,
          metadata={
              "help": "The model checkpoint for weights initialization. Leave None if you want to train a model from scratch."
          },

      )
      model_type: Optional[str] = field(
          default=None,
          metadata={
              "help": "If training from scratch, pass a model type from the list: "
              + ", ".join(MODEL_TYPES)
          },
      )
      cache_dir: Optional[str] = field(
          default=None,
          metadata={
              "help": "Where do you want to store the pretrained models downloaded from s3"
          },
      )


  @dataclass
  class DataTrainingArguments:
      """
      Arguments pertaining to what data we are going to input our model for training and eval.
      """

      train_data_file: Optional[str] = field(
          default=None, metadata={"help": "The input training data file (a text file)."}
      )
      eval_data_file: Optional[str] = field(
          default=None,
          metadata={
              "help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."
          },
      )
      line_by_line: bool = field(
          default=False,
          metadata={
              "help": "Whether distinct lines of text in the dataset are to be handled as distinct sequences."
          },
      )

      mlm: bool = field(
          default=False,
          metadata={
              "help": "Train with masked-language modeling loss instead of language modeling."
          },
      )

      block_size: int = field(
          default=-1,
          metadata={
              "help": "Optional input sequence length after tokenization."
              "The training dataset will be truncated in block of this size for training."
              "Default to the model max input length for single sentence inputs (take into account special tokens)."
          },
      )
      overwrite_cache: bool = field(
          default=False,
          metadata={"help": "Overwrite the cached training and evaluation sets"},
      )


  # Create LineByLineDataset from Movie Plots text file
  def get_dataset(
      args: DataTrainingArguments, tokenizer: PreTrainedTokenizer, evaluate=False
  ):
      file_path = args.eval_data_file if evaluate else args.train_data_file
      if args.line_by_line:
          return LineByLineTextDataset(
              tokenizer=tokenizer, file_path=file_path, block_size=args.block_size
          )
      else:
          return TextDataset(
              tokenizer=tokenizer,
              file_path=file_path,
              block_size=args.block_size,
              overwrite_cache=args.overwrite_cache,
          )

  def main():

      model_args = ModelArguments(
          model_name_or_path="/content/drive/MyDrive/Colab Notebooks/webpage/story_generator_checkpoint", model_type="gpt2-medium"
          # , cache_dir=""
      )
      data_args = DataTrainingArguments(
          train_data_file="/content/train_text1.txt",
          eval_data_file="/content/eval_text1.txt",
          line_by_line=True,
          block_size=512,
          overwrite_cache=True,
      )
      training_args = TrainingArguments(
          output_dir="story_generator_checkpoint",
          overwrite_output_dir=True,
          do_train=True,
          do_eval=True,
          # evaluate_during_training=False,
          logging_steps=500,
          per_device_train_batch_size=2,
          num_train_epochs=2,
          save_total_limit=1,
          save_steps=1000,
          learning_rate=2e-5,
          per_device_eval_batch_size=2,
          weight_decay=0.01,
          load_best_model_at_end=True,
      )

      if data_args.eval_data_file is None and training_args.do_eval:
          raise ValueError(
              "Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
              "or remove the --do_eval argument."
          )

      if (
          os.path.exists(training_args.output_dir)
          and os.listdir(training_args.output_dir)
          and training_args.do_train
          and not training_args.overwrite_output_dir
      ):
          raise ValueError(
              f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
          )

      # Setup logging
      logging.basicConfig(
          format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
          datefmt="%m/%d/%Y %H:%M:%S",
          level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
      )
      logger.warning(
          "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
          training_args.local_rank,
          training_args.device,
          training_args.n_gpu,
          bool(training_args.local_rank != -1),
          training_args.fp16,
      )
      logger.info("Training/evaluation parameters %s", training_args)

      # Set seed for deterministic training runs
      set_seed(training_args.seed)


      config = AutoConfig.from_pretrained(
          model_args.model_name_or_path, cache_dir=model_args.cache_dir
      )

      tokenizer = AutoTokenizer.from_pretrained(
          model_args.model_name_or_path, cache_dir=model_args.cache_dir
      )

      model = GPT2LMHeadModel.from_pretrained(
          model_args.model_name_or_path,
          from_tf=bool(".ckpt" in model_args.model_name_or_path),
          config=config,
          cache_dir=model_args.cache_dir,
      )

      special_tokens_dict = {
          "bos_token": "<BOS>",
          "eos_token": "<EOS>",
          "pad_token": "<PAD>",
          "additional_special_tokens": [
              "<superhero>",
              "<action>",
              "<drama>",
              "<thriller>",
              "<horror>",
              "<comedy>",
              "<sci_fi>"
          ],
      }

      num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
      model.resize_token_embeddings(len(tokenizer))

      if data_args.block_size <= 0: 
        # If block_size <= 0, set it to max. possible value allowed by model
          data_args.block_size = tokenizer.model_max_length
      else:
          data_args.block_size = min(data_args.block_size, tokenizer.model_max_length)

      # Get datasets

      train_dataset = (
          get_dataset(data_args, tokenizer=tokenizer) if training_args.do_train else None
      )
      eval_dataset = (
          get_dataset(data_args, tokenizer=tokenizer, evaluate=True)
          if training_args.do_eval
          else None
      )
      data_collator = DataCollatorForLanguageModeling(
          tokenizer=tokenizer,
          mlm=data_args.mlm,
      )

      # Initialize our Trainer
      trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          # prediction_loss_only=True,
      )




  # Run these cells for story generation
  from transformers import pipeline, TextGenerationPipeline, GPT2LMHeadModel, AutoTokenizer
  checkpoint = "/content/drive/MyDrive/Colab Notebooks/webpage/story_generator_checkpoint"

  model = GPT2LMHeadModel.from_pretrained(checkpoint)
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  story_generator = TextGenerationPipeline(model=model, tokenizer=tokenizer)
  # The format for input_prompt: "<BOS> <genre> Optional text..."
  # Supported genres: superhero, sci_fi, horror, thriller, action, drama , comedy





  # input_prompt = "<BOS> <superhero> Green Arrow "

  genre="<BOS> "+genreList
  input_list=[f1,f2,f3]
  extra=200
  firstInput=genre+" "+input_list[0]

  for i in range(len(input_list)):
    if (i==0): 
      story = story_generator(firstInput, max_length=300, do_sample=True,
                  repetition_penalty=1.2, temperature=1.5, 
                  top_p=0.95, top_k=50)
      lst_str = str(story)[36:-2] #38
      lst_str.replace('\\','')
      lst_str.replace(':','')
      lst_str.replace('/','')
      lst_str.replace('[citation needed]','')
      lst_str.replace('\\\'s',"'s")
      lst_str.replace("\\\\\\'s","'s")
      lst_str.replace("\\'s","'s")
      lst_str.replace("\'s","'s")
      lst_str=''.join(filter(lambda x: x!=' ’ \n' and x!='"\n' and x!='\n' and x!='’\n' and x!=' \n' and x!="\\\\\\", lst_str))
    else:
      extra+=100
      inputPrmpt=genre+" "+lst_str+input_list[i]
      story = story_generator(inputPrmpt, max_length=300+extra, do_sample=True,
                  repetition_penalty=1.2, temperature=1.5, 
                  top_p=0.95, top_k=50)
      lst_str = str(story)[40:-3]
      lst_str.replace('/','')
      lst_str.replace('[citation needed]' and '\\\'s' ,'')
      lst_str.replace("\\\\\\'s","'s")
      lst_str.replace("\\'s","'s")
      lst_str.replace("\'s","'s")
      lst_str.replace("\'","'s")
      lst_str.replace("\\\'","")
      lst_str.replace("\\xa0all",'')
      lst_str.replace("xa0all",'')
      lst_str.replace("xa0w",'')
      lst_str.replace('\\\\\\','')
      lst_str.replace("\ ","")
      lst_str.replace("\\\'t","'t")
      # print(lst_str)
      for i in lst_str:
         if (lst_str.find('\\')!=-1 ):
           lst_str.replace('\\','')
      lst_str=''.join(filter(lambda x: x!=' ’ \n' and x!='"\n' and x!='\n' and x!='’\n' and x!=' \n' and x!="\\\\\\", lst_str))



    # nextStr=genre+" "+lst_str
    
  # print(lst_str,sep='\n')


  # story = story_generator(input_prompt, max_length=300, do_sample=True,
  #                repetition_penalty=1.1, temperature=1.1, 
  #                top_p=0.95, top_k=50)

  # print(story)


  splitted=re.split('; |, |\*|\n',lst_str)
  splitted

  sentence_vec1=modelBert.encode(splitted)
  # print(sentence_vec1)
  encodedSentence1=[]
  inputSentence1=[s1,s2,s3]





  i=0
  while i<len(inputSentence1):
    encodedSentence1.insert(i,modelBert.encode(inputSentence1[i]))
    i+=1


  i = 0
  # k=0
  list_of_probabilities1=[]
  list_of_indicies1=[]
  for z in range(len(inputSentence1)):
    while i < len(splitted):
      list_of_probabilities1.append(cosine_similarity([sentence_vec1[i]],[encodedSentence1[z]]))
      i += 1
    lf1 = [val for sublist in list_of_probabilities1 for val in sublist]
    lf3=[val for sublist in lf1 for val in sublist]
    # print(lf3)
    x = max(lf3)
    list_of_indicies1.append(str(list_of_probabilities1.index(x)))
    # print("Highest Probability equals::")
    # print(x)
    # print("Index::" + str(list_of_probabilities1.index(x)))
    # print("Sentence::" + splitted[list_of_probabilities1.index(x)])
    #re initiallizing for next sentence
    list_of_probabilities1=[]
    i=0




  for i in range(len(inputSentence1)):
    splitted.insert(int(list_of_indicies1[i])+1,inputSentence1[i])


  finalStry=' '.join(splitted)
  return finalStry

In [ ]:
genGpt2('Superman wanted to save the world','Superman had to kill lex luthor','Lex luthor','<superhero>','superman was happy','superman loved his life','superman saved earth')

In [ ]:
#  # Training
#     try:
#       if training_args.do_train:
#           model_path = (
#               model_args.model_name_or_path
#               if model_args.model_name_or_path is not None
#               and os.path.isdir(model_args.model_name_or_path)
#               else None
#           )
#           trainer.train(model_path=model_path)
#           trainer.save_model()
#           tokenizer.save_pretrained(training_args.output_dir)
#     except KeyboardInterrupt:
#       print("Saving model that was in the middle of training")
#       trainer.save_model()
#       tokenizer.save_pretrained(training_args.output_dir)
#       return

#     # Evaluation
#     results = {}
#     if training_args.do_eval:
#         logger.info("*** Evaluate ***")

#         eval_output = trainer.evaluate()

#         perplexity = math.exp(eval_output["eval_loss"])
#         result = {"perplexity": perplexity}

#         output_eval_file = os.path.join(training_args.output_dir, "eval_results_lm.txt")
#         if trainer.is_world_process_zero():
#             with open(output_eval_file, "w") as writer:
#                 logger.info("***** Eval results *****")
#                 for key in sorted(result.keys()):
#                     logger.info("  %s = %s", key, str(result[key]))
#                     writer.write("%s = %s\n" % (key, str(result[key])))

#         results.update(result)

#     return results

# if __name__ == "__main__":
#     main()




# EMBEDDING SENTENCES IN AN EXISTING STORY


In [217]:
def gen(s1,s2,s3,s4,s5):
  import os
  import re

  directory = '/content/drive/MyDrive/Colab Notebooks/webpage/KidsStoriesAnimals/'

  TempM=[]
  Temp=[]
  StryName=[]
  # i=0
  for filename in os.listdir(directory):
      if filename.endswith(".txt"):
          TempX=[]
          with open("/content/drive/MyDrive/Colab Notebooks/webpage/KidsStoriesAnimals"+'/'+filename,"r") as f:
            TempX = [line.replace('\n','') for line in f.readlines() ]
            TempX= [  line.split('.') for line in TempX]
            TempX= [val for sublist in TempX for val in sublist]
            TempX= [  line.split('?') for line in TempX]
            TempX= [val for sublist in TempX for val in sublist]
            TempX= list(filter(lambda x: x!=' ’ \n' and x!='"\n' and x!='\n' and x!='’\n' and x!=' \n', TempX))
            

          TempM.append(TempX)
          StryName.append(filename)
          continue
      else:
        continue  
  # TempM return each story as a list where each line is a string
  for i in TempM:
    if (len(i)>0):
      i.pop()
  # print(TempM)         
  StrpStry=[]
  for i in range(0,4):
    with open("/content/drive/MyDrive/Colab Notebooks/webpage/KidsStoriesAnimals/"+StryName[i], "r") as a_file:
        x=a_file.readlines()
        StrpStry.append(x)
  # print(StrpStry)  
  k=0
  flattened=[]
  while k<len(TempM):
    flattened.insert(k,TempM[k])

    k+=1


  inputSentenceFind=[s1,s2,s3,s4]
  input_animal=s5

  options=[]
  indx_of_options=[]
  stry_of_options=[]

  for i in range(len(StryName)-1):
    if (StryName[i].find(input_animal) != -1):
      options.append(StryName[i])


  for i in range(len(options)):
    indx_of_options.append(StryName.index(options[i]))
    stry_of_options.append(flattened[indx_of_options[i]])


  max_of_prob=[]

  def iter(w):
    l=0
    i = 0
    list_of_probabilitiesFind=[]
    list_of_indiciesFind=[]
    list_of_summation=[]
    Sum=0
    encodedSentenceFind=[]


    while l<len(inputSentenceFind):
      encodedSentenceFind.insert(l,modelBert.encode(inputSentenceFind[l]))
      l+=1

    sentence_vecFind=modelBert.encode(stry_of_options[w]) 
    for z in range(len(inputSentenceFind)):
      while i < len(stry_of_options[w]):
        list_of_probabilitiesFind.append(cosine_similarity([sentence_vecFind[i]],[encodedSentenceFind[z]]))
        i += 1

        lfFind = [val for sublist in list_of_probabilitiesFind for val in sublist]
        lf2Find=[val for sublist in lfFind for val in sublist]
        Sum=sum(lf2Find)
      
      

    list_of_summation.insert(i,Sum)
    max_of_prob.append(Sum)
    # print(list_of_summation)



  for k in range(len(options)):
    iter(k)
      
  # Finding the matching story name and its index in our files
  maximum=max(max_of_prob)
  Indx=max_of_prob.index(maximum)
  # print('\n')
  # print("Index::",Indx)
  # print('\n')
  # print(options[Indx])   

  # Finding Where to place the sentences
  Story=stry_of_options[Indx]
  sentence_vec=modelBert.encode(Story) 
  encodedSentence=[]
  # inputSentence=["The lion is the strongest animal.","The lion wieghts 40kg","Nearly all wild lions live in Africa.","Lions are the second largest big cat species in the world "]
  i=0
  while i<len(inputSentenceFind):
    encodedSentence.insert(i,modelBert.encode(inputSentenceFind[i]))
    i+=1

  # Finding where to place inside the story contd.
  u = 0
  list_of_probabilities=[]
  list_of_indicies=[]
  for z in range(len(inputSentenceFind)):
    while u < len(Story):
      list_of_probabilities.append(cosine_similarity([sentence_vec[u]],[encodedSentence[z]]))
      u += 1

    lf = [val for sublist in list_of_probabilities for val in sublist]
    lf2=[val for sublist in lf for val in sublist]
    x = max(lf2)
    list_of_indicies.append(str(list_of_probabilities.index(x)))
    # print("Highest Probability equals::")
    # print(x)
    # print("Index::" + str(list_of_probabilities.index(x)))
    # print("Sentence::" + Story[list_of_probabilities.index(x)])
    #re initiallizing for next sentence
    list_of_probabilities=[]
    u=0

  # Placing the Sentences and Printing the output
  list_for_quotes=[]
  list_for_quotes2=[]
  StoryX=' '.join(Story)
  matches=re.findall(r'\“(.+?)\”',StoryX)
  matches2=re.findall(r'\"(.+?)\"',StoryX)
#--------------------------------------------- 
  for j in Story:
    for k in matches:
      
      if j.find(k) != -1:
        # print(j)
        # print(Story[3])
        list_for_quotes.append(Story.index(j))
# --------------------------------------------
  for j in Story:
    for k in matches2:
      
      if j.find(k) != -1:
        list_for_quotes2.append(Story.index(j))      
# --------------------------------------------
  # print(matches)      
  def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]  

  def second_largest(numbers):
      count = 0
      m1 = m2 = float('-inf')
      for x in numbers:
          count += 1
          if x > m2:
              if x >= m1:
                  m1, m2 = x, m1            
              else:
                  m2 = x
      return m2 if count >= 2 else None

  j=-1
  X=[]
  X1=[]
  X2=[] 
  X3=[]
  
  array_of_indx=[]
  array_of_indx1=[]
  array_of_indx2=[]
  array_of_indx3=[]


  w0=inputSentenceFind[0]
  w1=inputSentenceFind[1]
  w2=inputSentenceFind[2]
  w3=inputSentenceFind[3]
  
  for j in Story:
    x0=cosine_sim(j,w0)
    x1=cosine_sim(j,w1)
    x2=cosine_sim(j,w2)
    x3=cosine_sim(j,w3)
    
    X.append(x0)
    X1.append(x1)
    X2.append(x2)
    X3.append(x3)


  for i in X:
    if i !=0:
       array_of_indx.append(X.index(i))
       
  for i in X1:
    if i !=0:
       array_of_indx1.append(X1.index(i))       
  for i in X2:
    if i !=0:
       array_of_indx2.append(X2.index(i))
  for i in X3:
    if i !=0:
       array_of_indx3.append(X3.index(i))      

  IndexOfMaximumFromX=X.index(max(X))
  IndexOfMaximumFromX1=X1.index(max(X1))
  IndexOfMaximumFromX2=X2.index(max(X2))
  IndexOfMaximumFromX3=X3.index(max(X3))

  
  X=list(filter(lambda x: x!= 0, X))
  X1=list(filter(lambda x: x!= 0, X1))
  X2=list(filter(lambda x: x!= 0, X2))
  X3=list(filter(lambda x: x!= 0, X3))
 
  # print(IndexOfMaximumFromX)

  scndMaxX=X.index(second_largest(X))
  scndMaxX1=X1.index(second_largest(X1))
  scndMaxX2=X2.index(second_largest(X2))
  scndMaxX3=X3.index(second_largest(X3))
  
  randList=[IndexOfMaximumFromX,scndMaxX]
  randList1=[IndexOfMaximumFromX1,scndMaxX1]
  randList2=[IndexOfMaximumFromX2,scndMaxX2]
  randList3=[IndexOfMaximumFromX3,scndMaxX3]
  n = random.randint(0,1)
  n1 = random.randint(0,1)
  n2 = random.randint(0,1)
  n3 = random.randint(0,3)

# --------------------------------------------------
  # List_of_insert_indices=[]
  # List_of_insert_indices.append(IndexOfMaximumFromX)
  # List_of_insert_indices.append(n)
  # List_of_insert_indices.append(n)
  # List_of_insert_indices.append(n)

  List_of_linking_words=['as',',also','furthermore','additionally','and']
  rand = random.randint(0,len(List_of_linking_words)-1)
  rand1 = random.randint(0,len(List_of_linking_words)-1)
  rand2 = random.randint(0,len(List_of_linking_words)-1)
  rand3 = random.randint(0,len(List_of_linking_words)-1)


  for i in range(len(inputSentenceFind)):
    if (i==0):
      Story.insert(randList[n]+1,(List_of_linking_words[rand]+" "+inputSentenceFind[i])+".")
    if (i==1):
      Story.insert(randList1[n1]+1,(List_of_linking_words[rand1]+" "+inputSentenceFind[i])+".")
    if (i==2):
      Story.insert(randList2[n2]+1,(List_of_linking_words[rand2]+" "+inputSentenceFind[i])+".")  
    if (i==3):
      Story.insert(array_of_indx3[n3]+1,(List_of_linking_words[rand3]+" "+inputSentenceFind[i])+".")  

  Story=' '.join(Story)
  return Story

In [218]:
gen('cats eat alot','cats sleep alot ','cats are friendly','cats dont eat starch','cat')

'Chloe was a little girl  She had no friends but loved animals, especially cats furthermore cats are friendly. and cats eat alot.  On her tenth birthday, her father took her to an animal shelter where there were many cats  “You can chose a cat to take home, Chloe,” said her father furthermore cats dont eat starch. as cats sleep alot .  Chloe was overjoyed! She hugged her father and said, “Oh! This is my best gift ever ” Chloe liked all the cats at the shelter, but fell in love with a particular cat who had big, bright eyes  “I’ll take her,” Chloe said and named her Star  In a few days time, Star became very naughty  When Chloe was around she was good, but the moment she left for school, Star got down to her tricks  She tore the curtains, clawed the seats, lapped up all the milk in the kitchen and often stole fish  Chloe’s parents were very angry  One day, they told Chloe that they would return Star to the shelter, if she did not learn to behave Chloe was very worried, she did not know 

In [ ]:
def finder(s1,s2,s3,s4):
  #cleaning
  cleanStrys = []
  import os
  import re

  directory = '/content/drive/MyDrive/Colab Notebooks/webpage/KidsStoriesAnimals'

  TempM=[]
  Temp=[]
  StryName=[]
  # i=0
  for filename in os.listdir(directory):
      if filename.endswith(".txt"):
          TempX=[]
          with open("/content/drive/MyDrive/Colab Notebooks/webpage/KidsStoriesAnimals"+'/'+filename,"r") as f:
            TempX = [line.replace('\n','') for line in f.readlines() ]
            TempX= [  line.split('.') for line in TempX]
            TempX= [val for sublist in TempX for val in sublist]
            TempX= [  line.split('?') for line in TempX]
            TempX= [val for sublist in TempX for val in sublist]
            TempX= list(filter(lambda x: x!=' ’ \n' and x!='"\n' and x!='\n' and x!='’\n' and x!=' \n', TempX))
            #  TempX= [ l for l in line.split('?') for line in TempX]
          TempM.append(TempX)
          StryName.append(filename)
          continue
      else:
        continue  

  for st in TempM:
    single_story = []
    for ln in st:
      clean_sentence = cleantext.clean(ln, all= True)
      single_story.append(clean_sentence)
    cleanStrys.append(single_story)


  # Cleaning input sentences
  inputSentenceFind=[s1,s2,s3,s4]
  cleanInput = []
  for i in inputSentenceFind:
    clean_sentence = cleantext.clean(i, all= True)
    cleanInput.append(clean_sentence)



  #make each text file into single string
  cleanStrysJoined = []
  for st in cleanStrys:
    str = ''.join(st)
    cleanStrysJoined.append(str)
    

  vectorizer1 = TfidfVectorizer()
  maX = -43253535324
  maxindx = 0
  i = 0
  scores = []
  print(StryName)
  for z in cleanStrysJoined:
    
    # print('i : ', i)
    tfidf = vectorizer1.fit_transform([cleanInput[0], z])
    score = ((tfidf * tfidf.T).A)[0,1]
    scores.append(score)
    # print(score)

    if score>maX:
      maX = score
      maxindx = i

    i += 1

  print(scores, maX, maxindx)
  print('SCORING :: ', StryName[maxindx])

In [ ]:
print(options)

['five-wild-dogs.txt', 'the-farmer-and-the-dog.txt']


In [ ]:


  # test
def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


print(cosine_sim('Cats eat alot', 'cats sleep alot'))
print(cosine_sim('a little bird', 'a little bird chirps'))
print(cosine_sim('a little bird', 'a big dog barks'))

0.5031026124151314
0.7092972666062738
0.0


# Running with Flask

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template ,request,url_for,redirect,Response

app=Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
  return render_template('index.html')
  

@app.route('/Generated',methods=["POST"])
def generate():
      ifc1=request.form["fact1"]
      ifc2=request.form["fact2"]
      ifc3=request.form["fact3"]
      ifc4=request.form["fact4"]
      animalT=request.form["animalName"]
      x=gen(ifc1,ifc2,ifc3,ifc4,animalT)

      return render_template('generated.html',f1=x)



@app.route('/GPT2',methods=['POST','GET'])
def gptGen():
  return render_template('/gpt2Gen.html')



@app.route('/StryGen',methods=["POST"])
def generateGPT():
  s1=request.form['s1']
  s2=request.form['s2']
  s3=request.form['s3']
  genreForm=request.form['genreName']
  e1=request.form['e1']
  e2=request.form['e2']
  e3=request.form['e3']
  # print(s1,s2,s3,e1,e2,e3,genreForm)
  x=genGpt2(s1,s2,s3,'<superhero>',e1,e2,e3)
  
  return render_template('generated.html',f1=x)
 



  
if __name__ =='__main__':
  app.run()  